#Notebook
In this noteboook, use the exported tiff files to
-find min max (excl. <-30 dB values)
-calculate percentage of -30 dB pixels over images
-scale them
-remove/filter the -30 dB values
-write to tfrec



## Load libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pathlib
import matplotlib.pyplot as plt
# import heapq, random
import pandas as pd

In [3]:
import pathlib

In [4]:
from sklearn.utils import shuffle



In [5]:
!pip install -q rasterio
import rasterio

     |████████████████████████████████| 19.3 MB 1.5 MB/s 


In [6]:
import tensorflow as tf

In [7]:
import random


# Defintions

## Load (definitions)

### load files

In [8]:
def subsplit(x):
  return x.split("_",1)[-1].split(".tif")[0]


def select_corresponding_files(files_x,files_y):
  id_x = []
  for i in range(len(files_x)): #X becomes same as y, dif sequence than in X
    id_temp = subsplit(files_x[i])
    id_x.append(id_temp)

  id_y = [] #longer list possibly
  for i in range(len(files_y)): 
    id_temp = subsplit(files_y[i])
    id_y.append(id_temp)

  temp = set(id_x)
  res = [i for i, val in enumerate(id_y) if val in temp]

  files_y_corresponding = []
  for j in res: 
     files_y_corresponding.append(files_y[j])
  return  files_y_corresponding


In [9]:
# def get_tile_nr_y(y):


def get_tile_nr(x):
  return x.split("_",1)[-1].split(".tif")[0] 

def remove_duplicates_x_list(files_x):
  x_list = files_x.tolist()
  for item in x_list:
    tile_nr = get_tile_nr(item)
    if ("(" in item):
      print(item)
      x_list.remove(item)
  return x_list

def remove_duplicates_list_xy(files_x,files_y):
  x_list = files_x.tolist()
  for item in x_list:
    tile_nr = get_tile_nr(item)
    if ("(" in item):
      # print(item)
      x_list.remove(item)
  return x_list,files_y[0:len(x_list)]



def remove_duplicates_x(files_x):
  for item in files_x:
    tile_nr = get_tile_nr(item)
    if ("(" in item):
      # print(item)
      files_x.remove(item)
  return files_x



In [10]:

def get_files_ls(x_path, y_path):
  """"Make sure that the files x and y containing landslides are corresponding 
  """

  files_y = [str(f) for f in pathlib.Path(y_path).glob('*.tif')] #binary
  files_x = [str(f) for f in pathlib.Path(x_path).glob('*.tif')] #prepost
  
  files_x = remove_duplicates_x(files_x)
  files_y = remove_duplicates_x(files_y)

  len_y = len(files_y)
  len_x = len(files_x)

  files_x.sort(key = lambda x: int(subsplit(x)))
  files_y.sort(key = lambda x: int(subsplit(x)))

  if (len_x != len_y):
    files_y = select_corresponding_files(files_x,files_y)

  return files_x,files_y



In [11]:
# files_y = [str(f) for f in pathlib.Path(path_y1).glob('*.tif')] #binary
# files_x = [str(f) for f in pathlib.Path(path_x1).glob('*.tif')] #prepost

In [12]:
# len_y = len(files_y)
# len_x = len(files_x)
# print(len_y,len_x)

In [13]:
# x1,y1 = get_files_ls(path_x1,path_y1)

### open files

In [14]:
def open_tiff_x(x):
  """"" Open tiff file x. The sar 1 and sar 2 band was accidentally wrongly turned with exporting
  so that is now redone here. The output are the three bands of the x tiff image
  """
  raster = rasterio.open(x)
  #accidentally the 1st band was sar2, second band sar1
  sar = raster.read()
  return sar


In [15]:

def open_tiff_y(y):
  raster = rasterio.open(y)
  y = raster.read()
  # shape needs to be (256,256,1) (currently 1,256,256) Might depend per region what they have..
  y_moved = np.moveaxis(y, 0, -1) #np array, source  (axis), destination (axis)
  return y_moved


### divide into train , test

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
def split_train_test_val(x,y,test_size,val_size):
  """" Divide data into a train, val and test set 
  Train is for training model, val is for validating (and thus choosing) model during training,
  test is for testing the final model after training and thus qualifying the model
  For a division of 0.7,0.1,0.2 then use test size = 0.2, and then test_size = 0.125
  """
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=1)
  x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=val_size, random_state=1) # 0.125 x 0.8 = 0.1
  return x_train, y_train,x_test,y_test,x_val,y_val

In [18]:
def split_train_test(x,y,test_size):
  """" Divide data into a train, test set 
  Train is for training model,
  test is for testing the final model after training and thus qualifying the model
  """
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=1)
  return x_train, y_train,x_test,y_test

## Get min, max

In [19]:
# Remove -30 dB and lower  pixel values
#Save these sar bands to new npz
def get_min_max(x):
  
  #combine with min max?
  sqGridSar = []

  for i in range(len(x)):
    raster = rasterio.open(x[i])
    sar = raster.read()
    sqGridSar.append(sar)
 

  #Get min max
  max_sar_arr = np.nanmax(sqGridSar,axis = None)
  min_sar_arr = np.nanmin(sqGridSar,axis = None) #this should be -29!


  return min_sar_arr,max_sar_arr


  # Remove -30 dB and lower  pixel values
#Save these sar bands to new npz
def get_max(x):
  
  #combine with min max?
  sqGridSar = []

  for i in range(len(x)):
    raster = rasterio.open(x[i])
    sar = raster.read()
    sqGridSar.append(sar)
 

  #Get min max
  max_sar_arr = np.nanmax(sqGridSar,axis = None)

  return max_sar_arr

## scale values def

the min sar value can be put to -1, since this will always be -29 due to recommendation to filter out <-30
The -30 dB are also scaled to -1. 
The max sar value can be scaled to 0.5 or 0.8 or something like it

The max and min dem still need -0.8,0.8 or something WITHIN range of -1,1

The target y can be -1,1 obviously. Not sure if that is bad to have the target be -1,1 but the source -0.8,0.8 and another source band -1,0.8 :S

For only ls
des max sar is 26 and max dem is 85. The max sar comes from hiro, max dem comes from hu
asc max sar is 24 and max dem is 85. The max sar comes from hiro,max dem comes from hu

In [20]:
#function to map from left (original) to right (scaled)
def scaleGrid(input_grid,leftMin,leftMax, rightMin,rightMax):
  '''
  objective: map the values of the input grid to the proposed range
  input_grid: grid with values
  leftMin,leftMax: min and max of the original input grid
  rightMin,rightMax: min and max of the range to which the data is mapped  
  '''
 
  leftSpan = leftMax - leftMin
  rightSpan= rightMax - rightMin
 
  valueScaled = (input_grid - leftMin) / float(leftSpan)
  valueScaled = np.round(rightMin + (valueScaled * rightSpan),4)

 
  return valueScaled

In [21]:
def remove_nan(arr,fill):
  arr[np.isnan(arr)] = fill
  return arr

In [22]:
def scale_x(sar,min_sar,max_sar,max_value = 0.8):

  #scale X
  # min_sar = -29 
  sar = scaleGrid(sar,min_sar,max_sar,-0.8,max_value)
  sar = remove_nan(sar,fill = -1)
  return sar

In [23]:
def scale_y(y):
  fill = -1
  scaled_y = scaleGrid(y,0,255,-1,1)
  scaled_y = remove_nan(scaled_y,fill)
  return scaled_y


In [24]:
def to_np_float(arr):
  arr = np.float32(arr)
  return arr

In [25]:
def get_tile_nr(x):
  return x.split("_",1)[-1].split(".tif")[0] 

def remove_duplicates(files_x,files_y): #does this work well... Should be other way around to..
  for item in files_x:
    tile_nr = get_tile_nr(item)
    if ("(" in item):
      # print(item)
      files_x.remove(item)
  for item in files_y:
    tile_nr = get_tile_nr(item)
    if ("(" in item):
      files_y.remove(item)

  return files_x,files_y

## Convert to tfRecord

### Serialize example

In [26]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# def _int32_feature(value):
#   """Returns an int64_list from a bool / enum / int / uint."""
#   return tf.train.Feature(int32_list=tf.train.Int32List(value=[value]))

# def serialize_array(array):
#   array = tf.io.serialize_tensor(array)
#   return array

In [27]:

def serialize_example(path_x,path_y,min_sar,max_sar):

  #open y file
  y = open_tiff_y(path_y)
  y = scale_y(y)

  #open the x tiff file
  x = open_tiff_x(path_x)
  #scale x
  x = scale_x(x,min_sar,max_sar)

  
  #change all dtypes of array to float32, as float64 is not accepted by tfrec
  x = to_np_float(x)
  y = to_np_float(y)


  feature = {}
  feature['x'] = _bytes_feature(x.tobytes())
  feature['Y'] = _bytes_feature(y.tobytes())
    
  return tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()




add metadata

### Write

In [28]:
from tqdm.notebook import tqdm

In [29]:
def writeTFRecord(path_name,path_x,path_y,min_sar,max_sar,samples_per_tf):
  
  #path_x should be a [str(f) for f in path]
  # samples_per_tf = 100
  n_samples = len(path_x)


  splitNum = int(np.ceil(n_samples/samples_per_tf))

  for index in tqdm(range(splitNum)):
    partition_factor = np.round(n_samples/splitNum) #split number?
    path_x_part = path_x[int(partition_factor*index):int(partition_factor*(index+1))] #shape is 100,256,256
    path_y_part = path_y[int(partition_factor*index):int(partition_factor*(index+1))]

    record_file = path_name +"%d.tfrec" %index #index seems wrong, the name here at least!

    with tf.io.TFRecordWriter(record_file) as writer: 
      for i in tqdm(range(len(path_x_part))):
        writer.write(serialize_example(path_x_part[i],path_y_part[i],min_sar,max_sar))


In [30]:
def run_write(f_x,f_y,write_path,name_tfrec_file,min_sar,max_sar):
  """"" 
  f_x: tiff files of x images
  f_y : tiff files of y images
  write_path: path to local folder where tfrec shoudl be written to
  name_tfrec_files : name which has to end with _
  each tfrec file has a this name _ number, in such a way multiple tfrec files can be added to same kaggle dataset, otherwise names would be duplicate
  """""


 #if file already exists this gives error, just command this line then:
  # os.makedirs(write_path)
  # writeTFRecord(write_path+name_tfrec_file,f_x,f_y,100)
  writeTFRecord(write_path+name_tfrec_file,f_x,f_y,min_sar,max_sar,100)
                

In [31]:
def mix_regions(x1,x2,x3,x4,y1,y2,y3,y4):
  x = x1 + x2 + x3 + x4
  y = y1 + y2+ y3 + y4
  c = list(zip(x,y))
  random.shuffle(c)
  x,y = zip(*c)
  return x,y


In [32]:

def get_files(x_path,y_path):
  #get files x and y
  fx,fy = get_files_ls(x_path, y_path)
  #split intro train test val
  x_train, y_train,x_test,y_test,x_val,y_val = split_train_test_val(fx,fy,0.2,0.1)
  x_sets = [x_train,x_test,x_val]
  y_sets = [y_train,y_test,y_val]
  #get (min) max of x_train
  min_x,max_x = get_min_max(x_train)
  # max_x = get_max(x_train)
  return min_x,max_x,x_sets,y_sets

# Run functions

In [33]:
#files Y
path_y = "/content/drive/MyDrive/Thesis/Y/"
# path_y = path_files_og + "Y/" + "LS/"
!ls -la {path_y}

total 16
drwx------ 2 root root 4096 Oct  7 13:18 Hiro
drwx------ 2 root root 4096 Oct  7 13:18 Hok
drwx------ 2 root root 4096 Oct  7 13:18 Hu
drwx------ 2 root root 4096 Oct  7 13:46 Lom


In [34]:
#files X
path_x = "/content/drive/MyDrive/Thesis/ExportAratio/"
!ls -la {path_x}

total 16
drwx------ 2 root root 4096 Oct  7 13:45 Hiro
drwx------ 2 root root 4096 Oct  7 13:45 Hok
drwx------ 2 root root 4096 Oct  7 13:15 Hu
drwx------ 2 root root 4096 Oct  7 13:45 Lom


In [35]:
path_x1 = path_x + "Hu/"
path_x2 = path_x + "Hiro/"
path_x3 = path_x + "Hok/"
path_x4 = path_x + "Lom/"

path_y1 = path_y + "Hu/"
path_y2 = path_y + "Hiro/"
path_y3 = path_y + "Hok/"
path_y4 = path_y + "Lom/"

In [36]:
#mix the various regions.
min_x1,max_x1,x1,y1 = get_files(path_x1,path_y1)
# min_x1 #-15 ??

In [37]:


min_x2,max_x2,x2,y2 = get_files(path_x2,path_y2)


In [38]:
min_x3,max_x3,x3,y3 = get_files(path_x3,path_y3)

In [39]:
min_x4,max_x4,x4,y4 = get_files(path_x4,path_y4)

In [40]:



print(max_x1,max_x2,max_x2,max_x4)
print(min_x1,min_x2,min_x2,min_x4)
max_x = max([max_x1,max_x2,max_x2,max_x4])
min_x = min([min_x1,min_x2,min_x2,min_x4])
print(min_x,max_x)

train_x,train_y = mix_regions(x1[0],x2[0],x3[0],x4[0],y1[0],y2[0],y3[0],y4[0])
test_x,test_y = mix_regions(x1[1],x2[1],x3[1],x4[1],y1[1],y2[1],y3[1],y4[1])
val_x,val_y = mix_regions(x1[2],x2[2],x3[2],x4[2],y1[2],y2[2],y3[2],y4[2])

12.22187711929773 12.783960937574514 12.783960937574514 8.581809354082457
-15.520111171143919 -16.120684610798055 -16.120684610798055 -10.930583071874153
-16.120684610798055 12.783960937574514


In [41]:
tfrec_path = "/content/drive/MyDrive/Thesis/0710_tfrec_A_ratio_mask_08/"
os.makedirs(tfrec_path)
tfrec_path_train = tfrec_path + "train/"
tfrec_path_test = tfrec_path + "test/"
tfrec_path_val = tfrec_path + "val/"
os.makedirs(tfrec_path_train)
os.makedirs(tfrec_path_test)
os.makedirs(tfrec_path_val)

In [42]:
#convert to tfrecord (scale x, scale y incl.)
run_write(train_x,train_y,tfrec_path_train,'dif_train_ls',min_x,max_x)


  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

...

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

In [43]:
run_write(test_x,test_y,tfrec_path_test,'dif_test_ls',min_x,max_x)


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

In [44]:
run_write(val_x,val_y,tfrec_path_val,'dif_val_ls',min_x,max_x)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]